***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-4e69  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install --upgrade numpy
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install gensim==3.8.3

  Using cached numpy-1.24.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pointpats 2.2.0 requires opencv-contrib-python>=4.2.0, which is not installed.
scipy 1.6.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.1 which is incompatible.
pysal 2.4.0 requires urllib3>=1.26, but you have urllib3 1.25.11 which is incompatible.
  Using cached numpy-1.22.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.1
    Uninstalling numpy-1.24.1:
      Successfully uninstalled numpy-1.24.1
ERROR: pip's dependency resolver does not currently take into account all

In [3]:

import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from gensim.test.utils import common_texts
from gensim.models import Word2Vec, KeyedVectors  
from collections import defaultdict
import math
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 12 12:23 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
!pip install findspark
import findspark
findspark.init()

In [7]:
spark

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '207437625' 
full_path = f"gs://{bucket_name}/"
dir_n_title = "postings_gcp_title"
dir_n_anchor = "postings_gcp_anchor"
dir_n_text = "postings_gcp_body"
dir_n_pr ='PageRank'
dir_n_pv = 'Page_View'
dir_n_w2v = 'Word2Vec'
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

In [9]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
titles_list = {x[1]: x[0] for x in doc_title_pairs.collect()}

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import *

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

**word count**

In [14]:

def word_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [word for word in tokens if word not in all_stopwords]
    counts = Counter(tokens)
    tf_list = list()
    for word in counts:
        tf_list.append((word,(id,counts[word])))
    return tf_list


In [15]:
def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key=lambda x: x[0])

In [16]:
def fill_DL(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [x for x in tokens if x not in all_stopwords]
    return (id,len(tokens))

In [17]:
DL= doc_text_pairs.map(lambda x:fill_DL(x[0], x[1]))
DL=dict(DL.collect())

In [18]:
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_anchor=doc_anchor_pairs.map(lambda x:x[0])
word_counts_anchor=word_counts_anchor.flatMap(lambda x:x)
word_counts_anchor= word_counts_anchor.flatMap(lambda x:word_count(x[1], x[0]))

In [19]:
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_anchor = word_counts_anchor.groupByKey().mapValues(list)
postings_anchor =postings_anchor.map(lambda x : (x[0],sorted(list(set(x[1])))))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

### Document frequency and filteration

Next, we will filter out rare words, words that appear in 10 or fewer documents
 **(when working on the entire corpus, we will increase this threshold to a minimum of 50 documents)**

In [20]:
postings_text = postings_text.filter(lambda x: len(x[1])>50)

In [21]:
def calculate_df(postings):
    return postings.map(lambda x:(x[0],len(x[1])))

In [22]:
w2df_text_dict = calculate_df(postings_text)
w2df_text_dict = w2df_text_dict.collectAsMap()

w2df_anchor_dict = calculate_df(postings_anchor)
w2df_anchor_dict = w2df_anchor_dict.collectAsMap()

w2df_title_dict = calculate_df(postings_title)
w2df_title_dict = w2df_title_dict.collectAsMap()

In [25]:
def calc_doc_norm(text,id,N):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [word for word in tokens if word not in all_stopwords]
    count = Counter(tokens)
    tfidf = defaultdict()
    doc_len = len(tokens)
    for word,tf in count.items():
         if word in w2df_text_dict:##have more than 50 apperance
            tfidf[word] = (tf/doc_len)*(math.log(N/w2df_text_dict[word],10))
    norm =0
    for word,tfidf in tfidf.items():
        norm += tfidf**2
    return (id,math.sqrt(norm))

In [27]:
N = len(DL)
norm_text = doc_text_pairs.map(lambda x: calc_doc_norm(x[0], x[1],N))
norm_dict =dict(norm_text.collect()) 

## Partitioning and writing the index

In [37]:
NUM_BUCKETS = 124
def token2bucket_id(token,dir_n):
    if dir_n == "postings_gcp_body":
        return int(_hash(token),16) % NUM_BUCKETS
    if dir_n == "postings_gcp_anchor":
        return int(_hash(token),16) % NUM_BUCKETS +124
    if dir_n == "postings_gcp_title":
        return int(_hash(token),16) % NUM_BUCKETS +248

In [38]:
def partition_postings_and_write(postings,dir_n):
  # YOUR CODE HERE
    postings = postings.map(lambda x: (token2bucket_id(x[0],dir_n),x)).groupByKey().mapValues(list)
    return postings.map(lambda x: InvertedIndex(dir_n=dir_n).write_a_posting_list(x,bucket_name,dir_n))

In [ ]:
#`collect` operator to aggregate the posting location information from the multiple buckets/indices into a single dictionary
posting_locs_list_text = partition_postings_and_write(postings_text,dir_n_text).collect()
# merge the posting locations into a single dict 
super_posting_locs_text = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=dir_n_text):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_text[k].extend(v)

#`collect` operator to aggregate the posting location information from the multiple buckets/indices into a single dictionary
posting_locs_list_anchor = partition_postings_and_write(postings_anchor,dir_n_anchor).collect()
# merge the posting locations into a single dict 
super_posting_locs_anchor = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=dir_n_anchor):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_anchor[k].extend(v)

#`collect` operator to aggregate the posting location information from the multiple buckets/indices into a single dictionary
posting_locs_list_title = partition_postings_and_write(postings_title,dir_n_title).collect()
print(posting_locs_list_title)
# merge the posting locations into a single dict 
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=dir_n_title):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_title[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_text = InvertedIndex(dir_n = dir_n_text)
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs_text
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_text_dict
##add title-id list 
inverted_text.titles = titles_list
##add DL
inverted_text.DL = DL
# write the global statrite_indexs out
inverted_text.norm_dict = norm_dict
inverted_text.write_index('.', 'index_text')
# upload to gs
index_src = "index_text.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_text}/{index_src}'
!gsutil cp $index_src $index_dst

In [39]:
# Create inverted index instance
inverted_anchor = InvertedIndex(dir_n = dir_n_anchor)
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_anchor
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_anchor_dict
##add title-id list 
inverted_anchor.titles = titles_list
##add DL
inverted_anchor.DL = DL
# write the global stats out
inverted_anchor.write_index('.', 'index_anchor')
# upload to gs
index_src = "index_anchor.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_anchor}/{index_src}'
!gsutil cp $index_src $index_dst

[270, 250, 355, 295, 255, 340, 320, 325, 260, 305, 360, 315, 300, 290, 370, 345, 285, 265, 275, 330, 310, 280, 365, 335, 350, 301, 281, 331, 276, 306, 341, 351, 271, 316, 356, 371, 321, 326, 361, 256, 296, 336, 261, 266, 286, 251, 311, 291, 346, 366, 297, 307, 277, 347, 342, 367, 292, 282, 322, 267, 257, 272, 352, 327, 287, 332, 337, 362, 357, 252, 317, 302, 312, 262, 318, 353, 263, 313, 283, 298, 308, 368, 268, 358, 348, 333, 258, 253, 328, 273, 288, 303, 363, 338, 323, 278, 293, 248, 343, 254, 344, 274, 299, 349, 249, 364, 259, 369, 324, 359, 269, 354, 329, 319, 264, 334, 279, 289, 339, 314, 294, 309, 304, 284]


In [40]:
# Create inverted index instance
inverted_text = InvertedIndex(dir_n = dir_n_text)
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs_text
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_text_dict
##add title-id list 
inverted_text.titles = titles_list
##add DL
inverted_text.DL = DL
# write the global statrite_indexs out
inverted_text.norm_dict = norm_dict
inverted_text.write_index('.', 'index_text')
# upload to gs
index_src = "index_text.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_text}/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_text.pkl [Content-Type=application/octet-stream]...
/ [1 files][  6.0 MiB/  6.0 MiB]                                                
Operation completed over 1 objects/6.0 MiB.                                      


In [ ]:
# Create inverted index instance
inverted_anchor = InvertedIndex(dir_n = dir_n_anchor)
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_anchor
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_anchor_dict
##add title-id list 
inverted_anchor.titles = titles_list
##add DL
inverted_anchor.DL = DL
# write the global stats out
inverted_anchor.write_index('.', 'index_anchor')
# upload to gs
index_src = "index_anchor.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_anchor}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
# Create inverted index instance
inverted_title = InvertedIndex(dir_n = dir_n_title)
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_title_dict
##add title-id list 
inverted_title.titles = titles_list
##add DL
inverted_title.DL = DL 
# write the global stats out
inverted_title.write_index('.', 'index_title')
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_title}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
def generate_graph(pages):
    edges = pages.flatMapValues(lambda x :x).map(lambda x : (x[0],x[1].id)) .distinct()
    vertices = edges .flatMap(lambda x: x).distinct().map(lambda x : Row(id=x))
    return edges, vertices

In [ ]:
pages_links = parquetFile.select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr = pr.collect()
PR=defaultdict(float)
for row in pr:
    PR[row[0]]=row[1]
# pr.repartition(1).write.csv(f'gs://{bucket_name}/{dir_n_pr}/pr', compression="gzip")
# upload to gs
with open('PageRank.pkl', 'wb') as f:
    pickle.dump(PR, f)
index_src = "PageRank.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_pr}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp

In [ ]:
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
    for line in f:
        parts = line.split(' ')
        wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open("PageView.pkl", 'wb') as f:
    pickle.dump(wid2pv, f)
index_src = "PageView.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_pv}/{index_src}'
!gsutil cp $index_src $index_d


In [ ]:
with open("PageView.pkl", 'wb') as f:
    pickle.dump(wid2pv, f)
index_src = "PageView.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_pv}/{index_src}'
!gsutil cp $index_src $index_dst


In [ ]:
model = Word2Vec(sentences=common_texts, window=5, min_count=1, workers=4)

with open('Word2Vec.pkl', 'wb') as f:
    pickle.dump(model, f)
index_src = "Word2Vec.pkl"
index_dst = f'gs://{bucket_name}/{dir_n_w2v}/{index_src}'
!gsutil cp $index_src $index_dst